프로그래밍 언어에 대한 시를 쓰는 데 특화된 체인과 시를 설명하는 데 특화된 체인을 만드세요.

LCEL을 사용해 두 체인을 서로 연결합니다.

최종 체인은 프로그래밍 언어의 이름을 받고 시와 그 설명으로 응답해야 합니다.

모델로는 "gpt-3.5-turbo"를 사용하고 프롬프트에는 ChatPromptTemplate을 사용하세요.


In [6]:
# 필요한 라이브러리 import

import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# OpenAI API 키 설정 (실제 사용시 환경변수로 설정하세요)
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

print("✅ 라이브러리 import 완료!")

✅ 라이브러리 import 완료!


In [7]:
# 1. 프로그래밍 언어에 대한 시를 생성하는 체인

# 시 생성용 프롬프트 템플릿
poetry_prompt = ChatPromptTemplate.from_template(
    """
당신은 창의적인 시인입니다. 주어진 프로그래밍 언어에 대해 아름답고 감성적인 시를 써주세요.

프로그래밍 언어: {language}

요구사항:
- 4-6줄의 짧은 시로 작성
- 프로그래밍 언어의 특징이나 철학을 시적으로 표현
- 감성적이고 아름다운 언어 사용
- 한국어로 작성
- 맨 마지막엔 두줄 띄기
- 제목은 프로그래밍 언어의 이름을 따서 작성

Return:

시:

"""
)

# OpenAI 모델 설정 (gpt-5-nano 사용)
model = ChatOpenAI(
    model="gpt-5-nano",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 체인 구성 (LCEL 사용)
poetry_chain = poetry_prompt | model

In [8]:
# 2. 시에 대한 설명을 생성하는 체인

# 시 설명용 프롬프트 템플릿
explanation_prompt = ChatPromptTemplate.from_template(
    """
당신은 문학 비평가입니다. 주어진 시에 대해 전문적이고 깊이 있는 설명을 해주세요.

프로그래밍 언어: {language}
시: {poetry}

요구사항:
- 시의 주제와 메시지 분석
- 사용된 문학적 기법 설명
- 프로그래밍 언어와의 연관성 해석
- 시의 감정적 효과 설명
- 3-4문장으로 간결하게 작성
- 한국어로 작성

Return:

시 설명:

"""
)


# 체인 구성 (LCEL 사용)
explanation_chain = explanation_prompt | model

In [9]:
# 3. LCEL을 사용해 두 체인을 연결하는 최종 체인
combined_chain = {"poetry": poetry_chain, "language": RunnablePassthrough()} | explanation_chain

In [10]:
# 4. 체인 테스트

explanation = combined_chain.invoke("Python")

시:
Python
파이썬은 맑은 호수처럼 읽히는 마음의 언어다.
들여쓰기의 계단은 생각의 길을 밝히고, 공백은 숨을 준다.
읽기 쉬운 문법은 아이디어를 다리처럼 이어 주며, 복잡함을 덜어낸다.
동적 타이핑은 꿈을 자유롭게 흘려보내되, 의도는 선명하게 남는다.
아름다움이 추함보다 낫고, 간결함은 힘이 된다— 파이썬의 속삭임。


시 설명:
이 시는 파이썬의 문법을 마음의 언어로 비유하는 은유와 의인화를 통해 주제와 메시지를 제시한다. 들여쓰기의 계단과 공백은 사고의 흐름과 호흡을 구조화하는 이미지로, 간결함이 생각의 길을 밝힌다고 말한다. 동적 타이핑은 자유로운 꿈의 흐름을 허용하되 의도는 선명하다는 구도에서 기술 자유와 명확성의 균형을 강조한다. 전반적으로 파이썬의 속삭임은 아름다움과 간결함의 미학을 통해 감정적 여운을 남기며, 기술과 인간성 사이의 다리를 건너려는 마음을 자극한다.